In [2]:
import pandas as pd

In [3]:
df = pd.read_pickle('../data/processed/rumc.pkl')
df_rumc = pd.read_csv('../data/raw/covid_ct_CF_anon_with_date_shifts.csv')

In [7]:
df.columns.str.contains('phosphatase').any()

False

In [8]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [59]:
pd.Series(*zip(*[(similar('Urea', str(col)), col) for col in df.columns])).sort_values(ascending=False).head(20)

Uraat            0.666667
Ureum            0.666667
pH (Urine)       0.428571
Urinekweek       0.428571
Kreatinine       0.428571
Paracetamol      0.400000
IgG (Urine)      0.400000
corads           0.400000
Methotrexaat     0.375000
Borrelia IgM     0.375000
Borrelia IgG     0.375000
PISA ERO Area    0.352941
Brucella Bang    0.352941
Ureum (Urine)    0.352941
Eiwit (Urine)    0.352941
Uraat (Urine)    0.352941
TV Vmean         0.333333
MV Vmean         0.333333
PV Vmean         0.333333
AV Vmean         0.333333
dtype: float64

In [4]:
features = [
    'Alkalische fosfatase', 
    'ALAT', 
    'ASAT', 
    'Bilirubine direct', 
    'Bilirubine', 
    'Bilirubine totaal', 
    'CK', 
    'Basofielen', 
    'Eosinofielen', 
    'Eosinofiele granulocyten', 
    'Eosinofiele granulocyten EOSINOFIELE', 
    'Lymfocyten', 
    'Lymfocyten md', 
    'Monocyten', 
    'Monocyten md', 
    'D-Dimeer', 
    'Ferritine', 
    'Hemoglobine', 
    'Hematocriet', 
    'Kalium', 
    'Kreatinine', 
    'Lactaat', 
    'Leukocyten', 
    'Leucocyten', 
    'Magnesium', 
    'Natrium', 
    'Procalcitonine', 
    'Trombocyten', 
    'Ureum'
]

In [38]:
df_rumc[df_rumc.labresultsname.str.match(features[6]+r'\s*(?:\(.+?\))?', na=False)].labresultscommonname.str.extract(r'([^()]*[^\s()])(?:\s+\(.+?\))?').iloc[0, 0]

'CK'

In [87]:
mapping = [(f, df_rumc[df_rumc.labresultsname.str.match(f+r'\s*(?:\(.+?\))?', na=False)].labresultscommonname.str.extract(r'([^()]*[^\s()])(?:\s+\(.+?\))?').iloc[0, 0]) for f in features]

In [91]:
_, new_vals = zip(*mapping)
# print('\n- '.join(new_vals))
print(new_vals)

('ALKALISCHE FOSFATASE', 'ALAT', 'ASAT', 'BILIRUBINE DIRECT', 'BILIRUBINE DIRECT', 'BILIRUBINE TOTAAL', 'CK', 'BASOFIELEN', 'EOSINOFIELEN', 'EOSINOFIELE GRANULOCYTEN CHL', 'EOSINOFIELE GRANULOCYTEN MD', 'LYMFOCYTEN', 'LYMFOCYTEN MD', 'MONOCYTEN MD', 'MONOCYTEN MD', 'D-DIMEER', 'FERRITINE', 'HEMOGLOBINE', 'HEMATOCRIET', 'KALIUM', 'KREATININE', 'LACTAAT', 'LEUK', "LEUCO'S", 'MAGNESIUM', 'NATRIUM', 'PROCALCITONINE', 'TROMBOCYTEN', 'UREUM')


In [44]:
df_rumc[df_rumc.labresultsname == 'Leucocyten'].labresultscommonname.value_counts()

LEUCO'S    875
Name: labresultscommonname, dtype: int64

In [45]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [79]:
names = df_rumc.labresultscommonname.str.extract(r'([^()]*[^\s()])(?:\s+\(.+?\))?').value_counts()[df_rumc.labresultscommonname.str.extract(r'([^()]*[^\s()])(?:\s+\(.+?\))?').value_counts() > 12].index.values

In [83]:
names[0][0]

'GLUCOSE'

In [84]:
from itertools import combinations
similarities = []
for a, b in combinations(range(len(names)), 2):
    if a == b:
        continue
    name_a = names[a][0]
    name_b = names[b][0]
    s = similar(name_a, name_b)
    similarities.append({'a': name_a, 'b': name_b, 'similarity': s})
similarities = pd.DataFrame(similarities)

In [86]:
similarities.sort_values(by='similarity', ascending=False)[:30]

,a,b,similarity
45041,PCR CORONAVIRUS NCOV-19,PCR CORONAVIRUS NCOV19,0.977778
184368,TBNK T-CEL MARKER CD8+ BINNEN CD3 ABSOLUUT,TBNK T-CEL MARKER CD4+ BINNEN CD3 ABSOLUUT,0.976190
68035,FRC REBREATHING RV,FRC REBREATHING ERV,0.972973
44597,TRANSFUSIE-EENHEID VOOR RAPPORTAGE1,TRANSFUSIE-EENHEID VOOR RAPPORTAGE3,0.971429
73353,TRANSFUSIE-EENHEID VOOR RAPPORTAGE2,TRANSFUSIE-EENHEID VOOR RAPPORTAGE3,0.971429
183652,TBNK T-CEL MARKER CD8+ BINNEN CD3 %,TBNK T-CEL MARKER CD4+ BINNEN CD3 %,0.971429
189365,TRANSFUSIE-EENHEID VOOR RAPPORTAGE3,TRANSFUSIE-EENHEID VOOR RAPPORTAGE4,0.971429
44645,TRANSFUSIE-EENHEID VOOR RAPPORTAGE1,TRANSFUSIE-EENHEID VOOR RAPPORTAGE4,0.971429
73401,TRANSFUSIE-EENHEID VOOR RAPPORTAGE2,TRANSFUSIE-EENHEID VOOR RAPPORTAGE4,0.971429
42558,PLAVEISELEPITHEEL,PLAVEISEL EPITHEEL,0.971429


In [63]:
print('\n- '.join(features))

Alkalische fosfatase
- ALAT
- ASAT
- Bilirubine direct
- Bilirubine
- Bilirubine totaal
- CK
- Basofielen
- Eosinofielen
- Eosinofiele granulocyten
- Eosinofiele granulocyten EOSINOFIELE
- Lymfocyten
- Lymfocyten md
- Monocyten
- Monocyten md
- D-Dimeer
- Ferritine
- Hemoglobine
- Hematocriet
- Kalium
- Kreatinine
- Lactaat
- Leukocyten
- Leucocyten
- Magnesium
- Natrium
- Procalcitonine
- Trombocyten
- Ureum


In [58]:
df['Trombocyten'].isna().sum()

299